### Описание задания
Суть шифра Цезаря заключается в следующем: каждая буква алфавита заменяется на другую букву, смещённую на фиксированное число позиций вперёд или назад по алфавиту.

В этом случае реализуются следующие шаги шифрования:

* выбирается ключ (смещение), который определяет, насколько нужно сместить каждую букву.
* каждая буква исходного текста заменяется на ту, которая находится на выбранное количество позиций позже в алфавите.

### Задание:

1. определите алфавит, с которым будете работать (совет: если выбираете русский язык, исключите букву «ё»);
2. напишите алгоритм шифра Цезаря для генерации выборки (определите сдвиг на k каждой буквы, например, при сдвиге на 2 буква “А” переходит в букву “В” и так далее);
3. постройте нейронную сеть и обучите ее по принципу: вход - зашифрованная фраза, выход - дешифрованная фраза;
4. проверьте качество сети.

In [ ]:
import torch
import torch.nn as nn
import random

In [ ]:
# Выбрать алфавит
alphabet = 'АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
alphabet_size = len(alphabet)

### 1. Алгоритм шифра Цезаря

In [ ]:
# Шифровщик
def encrypt(text, k):
    encrypted_text = ''

    for char in text:
        if char in alphabet:
            index = alphabet.index(char)
            new_index = (index + k) % alphabet_size
            encrypted_text += alphabet[new_index]
        else:
            encrypted_text += char
    return encrypted_text

In [ ]:
# Дешифровщик
def decrypt(text, k):
    decrypted_text = ''

    for char in text:
        if char in alphabet:
            index = alphabet.index(char)
            new_index = (index - k) % alphabet_size
            decrypted_text += alphabet[new_index]
        else:
            decrypted_text += char
    return decrypted_text

### 2. Генерация датасета

In [ ]:
def generate_random_phrase(l=10):
    return ''.join(random.choices(alphabet, k=l))

def generate_dataset(n_samples, phrase_length=10, max_shift=5):
    dataset = []

    for _ in range(n_samples):
        phrase = generate_random_phrase(phrase_length)
        k = random.randint(1, max_shift)
        encrypted_phrase = encrypt(phrase, k)
        dataset.append((encrypted_phrase, phrase))
    return dataset

In [ ]:
# Преобразование символов в индексы и обратно
char_to_idx = {char: idx for idx, char in enumerate(alphabet)}
idx_to_char = {idx: char for idx, char in enumerate(alphabet)}

def phrase_to_tensor(phrase):
    return torch.tensor([char_to_idx[char] for char in phrase], dtype=torch.long)

def tensor_to_phrase(tensor):
    return ''.join([idx_to_char[idx.item()] for idx in tensor])

In [ ]:
def split_dataset(dataset, train_ratio=0.8):
    random.shuffle(dataset)
    split_index = int(len(dataset) * train_ratio)
    train_data = dataset[:split_index]
    val_data = dataset[split_index:]
    return train_data, val_data

In [ ]:
# Генерация датасета
dataset = generate_dataset(2000, phrase_length=6, max_shift=3)

# Разделить на обучающую и валидационную выборки
train_data, val_data = split_dataset(dataset, train_ratio=0.8)

In [ ]:
print('Примеры из датасета:')
for i, (encrypted, original) in enumerate(train_data[:10]):
    print(f'Пример {i+1}:\n   Зашифрованная фраза: {encrypted}\n   Исходная фраза: {original}\n')

Примеры из датасета:
Пример 1:
   Зашифрованная фраза: ННЕЫЗД
   Исходная фраза: ЛЛГЩЕВ

Пример 2:
   Зашифрованная фраза: НЮМЭТК
   Исходная фраза: ЛЬКЫРИ

Пример 3:
   Зашифрованная фраза: ЪИЖЙАС
   Исходная фраза: ШЖДЗЮП

Пример 4:
   Зашифрованная фраза: ЬЖХИЗШ
   Исходная фраза: ЫЕФЗЖЧ

Пример 5:
   Зашифрованная фраза: ЙДЕЬТК
   Исходная фраза: ЖБВЩПЗ

Пример 6:
   Зашифрованная фраза: ЩЛБЯМЯ
   Исходная фраза: ЦИЮЬЙЬ

Пример 7:
   Зашифрованная фраза: ФДЩУХЩ
   Исходная фраза: СБЦРТЦ

Пример 8:
   Зашифрованная фраза: ЙГАЧКШ
   Исходная фраза: ИВЯЦЙЧ

Пример 9:
   Зашифрованная фраза: ЗКЧФВГ
   Исходная фраза: ЕИХТАБ

Пример 10:
   Зашифрованная фраза: НСУЮХН
   Исходная фраза: КОРЫТК



### 3. Создать рекурентную сеть

In [ ]:
class CryptModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CryptModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_out, _ = self.rnn(embedded)
        output = self.fc(rnn_out)
        return output

In [ ]:
# Параметры модели
input_size = alphabet_size
hidden_size = 64
output_size = alphabet_size
batch_size = 32
n_epochs = 100
learning_rate = 0.001

In [ ]:
# Создать экземпляр модели
model = CryptModel(input_size, hidden_size, output_size)

### 4. Настройка оптимизатора и функции потерь

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### 5. Обучение и оценка модели

In [ ]:
def evaluate(model, val_data):
    model.eval()
    total_correct = 0
    total_chars = 0

    with torch.no_grad():
        for encrypted, original in val_data:
            encrypted_tensor = phrase_to_tensor(encrypted).unsqueeze(0)
            original_tensor = phrase_to_tensor(original)

            outputs = model(encrypted_tensor)
            _, predicted = torch.max(outputs, dim=2)

            total_correct += (predicted.squeeze(0) == original_tensor).sum().item()
            total_chars += len(original)

    accuracy = (total_correct / total_chars) * 100
    return accuracy

In [ ]:
def train(model, train_data, val_data, n_epochs, criterion, optimizer, batch_size):
    for epoch in range(n_epochs):
        model.train()
        random.shuffle(train_data)
        total_loss = 0

        # Обучение на обучающей выборке
        for i in range(0, len(train_data), batch_size):
            batch = train_data[i:i + batch_size]
            encrypted_batch, original_batch = zip(*batch)

            encrypted_tensors = [phrase_to_tensor(e) for e in encrypted_batch]
            original_tensors = [phrase_to_tensor(o) for o in original_batch]

            # Дополнение последовательностей до одинаковой длины
            encrypted_tensors = torch.nn.utils.rnn.pad_sequence(encrypted_tensors, batch_first=True)
            original_tensors = torch.nn.utils.rnn.pad_sequence(original_tensors, batch_first=True)

            # Прямой проход
            outputs = model(encrypted_tensors)  # (batch_size, seq_len, output_size)
            loss = criterion(outputs.view(-1, output_size), original_tensors.view(-1))

           # Обратный проход с оптимизатором
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Оценка на валидационной выборке
        val_accuracy = evaluate(model, val_data)

        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss:.3f}, Val Accuracy: {val_accuracy:.3f}')

In [ ]:
# Обучить модель
train(
    model=model,
    train_data=train_data,
    val_data=val_data,
    n_epochs=n_epochs,
    criterion=criterion,
    optimizer=optimizer,
    batch_size=batch_size
)

Epoch [1/100], Loss: 147.077, Val Accuracy: 33.208
Epoch [2/100], Loss: 97.197, Val Accuracy: 33.292
Epoch [3/100], Loss: 71.043, Val Accuracy: 34.417
Epoch [4/100], Loss: 62.594, Val Accuracy: 33.375
Epoch [5/100], Loss: 59.494, Val Accuracy: 35.125
Epoch [6/100], Loss: 57.939, Val Accuracy: 32.667
Epoch [7/100], Loss: 56.880, Val Accuracy: 32.417
Epoch [8/100], Loss: 56.171, Val Accuracy: 33.083
Epoch [9/100], Loss: 55.711, Val Accuracy: 34.125
Epoch [10/100], Loss: 55.213, Val Accuracy: 31.583
Epoch [11/100], Loss: 54.837, Val Accuracy: 33.333
Epoch [12/100], Loss: 54.541, Val Accuracy: 33.458
Epoch [13/100], Loss: 54.178, Val Accuracy: 33.500
Epoch [14/100], Loss: 53.913, Val Accuracy: 34.250
Epoch [15/100], Loss: 53.669, Val Accuracy: 34.083
Epoch [16/100], Loss: 53.430, Val Accuracy: 34.875
Epoch [17/100], Loss: 53.083, Val Accuracy: 33.167
Epoch [18/100], Loss: 52.871, Val Accuracy: 33.583
Epoch [19/100], Loss: 52.635, Val Accuracy: 33.333
Epoch [20/100], Loss: 52.441, Val Accur

In [ ]:
# Сохранить модель
torch.save(model.state_dict(), 'crypt_model.pth')

### 6. Протестировать модель

In [ ]:
def decrypt_with_model(model, encrypted_phrase):
    encrypted_tensor = phrase_to_tensor(encrypted_phrase).unsqueeze(0)
    with torch.no_grad():
        outputs = model(encrypted_tensor)
        _, predicted = torch.max(outputs, dim=2)
    return tensor_to_phrase(predicted.squeeze(0))

In [ ]:
def test_model(model, n_tests=5):
    print("Тестирование модели:")
    for i in range(n_tests):
        test_phrase = generate_random_phrase(10)
        k = random.randint(1, 5)
        encrypted_test_phrase = encrypt(test_phrase, k)

        predicted_phrase = decrypt_with_model(model, encrypted_test_phrase)

        print(f'Тест {i+1}:\n  Исходная фраза: {test_phrase}\n  Зашифрованная фраза: {encrypted_test_phrase}\n  Расшифрованная фраза: {predicted_phrase}\n')

In [ ]:
# Загрузка модели
loaded_model = CryptModel(input_size, hidden_size, output_size)
loaded_model.load_state_dict(torch.load("crypt_model.pth"))
loaded_model.eval()

# Тестирование загруженной модели
test_model(loaded_model, n_tests=10)

Тестирование модели:
Тест 1:
  Исходная фраза: АБЧЯЫЙШУДЮ
  Зашифрованная фраза: ДЕЫГЯНЬЧИВ
  Расшифрованная фраза: БВЩАЬКЪЦЖА

Тест 2:
  Исходная фраза: ЛОСВЮХХВЬЭ
  Зашифрованная фраза: ПТХЖВЩЩЖАБ
  Расшифрованная фраза: НПФЕБЦЦГЮА

Тест 3:
  Исходная фраза: ХМАЪЧНРЖЧН
  Зашифрованная фраза: ЪСЕЯЬТХЛЬТ
  Расшифрованная фраза: ШРВЮЪСУЙЪС

Тест 4:
  Исходная фраза: ЭАОЛЧЖВОЪЙ
  Зашифрованная фраза: БДТПЫКЖТЮН
  Расшифрованная фраза: ЮБРОЪЙЕРЫК

Тест 5:
  Исходная фраза: БЭХНВЮЮУШЪ
  Зашифрованная фраза: ВЮЦОГЯЯФЩЫ
  Расшифрованная фраза: АЫХНВЭЮТЦШ

Тест 6:
  Исходная фраза: ПЪТЗПЮЯЬЯД
  Зашифрованная фраза: СЬФЙСАБЮБЖ
  Расшифрованная фраза: ПЫТЖОЯЮЫЮГ

Тест 7:
  Исходная фраза: ЦХФЗМЬТИБН
  Зашифрованная фраза: ЧЦХИНЭУЙВО
  Расшифрованная фраза: ЦХТЗКЫРЖАН

Тест 8:
  Исходная фраза: ПЬФДЪЧФНЮС
  Зашифрованная фраза: ФБЩЙЯЬЩТГЦ
  Расшифрованная фраза: ТЮШИЮЫЦПВУ

Тест 9:
  Исходная фраза: ЗЬХКЭЧОЖКЕ
  Зашифрованная фраза: ЛАЩОБЫТКОЙ
  Расшифрованная фраза: ЙЭШЛЮШСЙНЗ

Тест 10:
  Исход

Как-то с реккурентной сетью не получилось, возможно она слишком сложная для этого простого действия, поэтому попробовала обычную архитектуру полносвязной нейросети

In [ ]:
class CryptModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CryptModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        embedded = self.embedding(x)
        x = self.fc1(embedded)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [ ]:
# Параметры модели
input_size = alphabet_size
hidden_size = 64
output_size = alphabet_size
batch_size = 32
n_epochs = 100
learning_rate = 0.001

# Создание экземпляра модели
model = CryptModel(input_size, hidden_size, output_size)

# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def evaluate(model, val_data):
    model.eval()
    total_correct = 0
    total_chars = 0

    with torch.no_grad():
        for encrypted, original in val_data:
            encrypted_tensor = phrase_to_tensor(encrypted).unsqueeze(0)
            original_tensor = phrase_to_tensor(original)

            outputs = model(encrypted_tensor)
            _, predicted = torch.max(outputs, dim=2)

            total_correct += (predicted.squeeze(0) == original_tensor).sum().item()
            total_chars += len(original)

    accuracy = (total_correct / total_chars) * 100
    return accuracy

In [ ]:
def train(model, train_data, val_data, n_epochs, criterion, optimizer, batch_size):
    for epoch in range(n_epochs):
        model.train()
        random.shuffle(train_data)
        total_loss = 0

        # Обучение на обучающей выборке
        for encrypted, original in train_data:
            encrypted_tensor = phrase_to_tensor(encrypted).unsqueeze(0)
            original_tensor = phrase_to_tensor(original)

            # Прямой проход
            outputs = model(encrypted_tensor)  # (1, seq_len, output_size)
            loss = criterion(outputs.view(-1, output_size), original_tensor)

            # Обратный проход с оптимизатором
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        # Оценка на валидационной выборке
        val_accuracy = evaluate(model, val_data)

        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss:.3f}, Val Accuracy: {val_accuracy:.3f}')

In [ ]:
train(
    model=model,
    train_data=train_data,
    val_data=val_data,
    n_epochs=n_epochs,
    criterion=criterion,
    optimizer=optimizer,
    batch_size=batch_size
)

# Сохранить модель
torch.save(model.state_dict(), 'crypt_model2.pth')

Epoch [1/100], Loss: 5151.608, Val Accuracy: 33.667
Epoch [2/100], Loss: 5074.865, Val Accuracy: 33.458
Epoch [3/100], Loss: 5073.173, Val Accuracy: 33.458
Epoch [4/100], Loss: 5072.470, Val Accuracy: 33.167
Epoch [5/100], Loss: 5065.206, Val Accuracy: 33.167
Epoch [6/100], Loss: 5069.958, Val Accuracy: 33.833
Epoch [7/100], Loss: 5062.104, Val Accuracy: 33.583
Epoch [8/100], Loss: 5059.570, Val Accuracy: 33.833
Epoch [9/100], Loss: 5057.343, Val Accuracy: 33.833
Epoch [10/100], Loss: 5058.002, Val Accuracy: 33.833
Epoch [11/100], Loss: 5058.124, Val Accuracy: 33.833
Epoch [12/100], Loss: 5054.958, Val Accuracy: 33.583
Epoch [13/100], Loss: 5052.470, Val Accuracy: 33.542
Epoch [14/100], Loss: 5052.962, Val Accuracy: 33.375
Epoch [15/100], Loss: 5052.359, Val Accuracy: 33.375
Epoch [16/100], Loss: 5051.798, Val Accuracy: 33.458
Epoch [17/100], Loss: 5051.920, Val Accuracy: 33.542
Epoch [18/100], Loss: 5050.741, Val Accuracy: 33.542
Epoch [19/100], Loss: 5051.425, Val Accuracy: 33.542
Ep

In [ ]:
# Загрузка модели
loaded_model = CryptModel(input_size, hidden_size, output_size)
loaded_model.load_state_dict(torch.load("crypt_model2.pth"))
loaded_model.eval()

# Тестирование загруженной модели
test_model(loaded_model, n_tests=10)

Тестирование модели:
Тест 1:
  Исходная фраза: РОВЩЗНМЭЖФ
  Зашифрованная фраза: ФТЖЭЛСРБКШ
  Расшифрованная фраза: ТРДЪЙПОЮЙЧ

Тест 2:
  Исходная фраза: ДЧЫЬЙЛЮКЦЭ
  Зашифрованная фраза: ЖЩЭЮЛНАМШЯ
  Расшифрованная фраза: ДЧЪЫЙМЯЙЧЭ

Тест 3:
  Исходная фраза: ШЬЙЗЙЖЕГХЖ
  Зашифрованная фраза: ЬАНЛНКЙЗЩК
  Расшифрованная фраза: ЪЯМЙМЙИДЧЙ

Тест 4:
  Исходная фраза: ЖЭЙЮСЬСЧМВ
  Зашифрованная фраза: ИЯЛАУЮУЩОД
  Расшифрованная фраза: ЧЭЙЯРЫРЧМГ

Тест 5:
  Исходная фраза: ОЪИФМАЙРЛШ
  Зашифрованная фраза: ПЫЙХНБКСМЩ
  Расшифрованная фраза: ОЪИЬМЮЙПЙЧ

Тест 6:
  Исходная фраза: РШФЭЫЖИНСЦ
  Зашифрованная фраза: УЫЧАЮЙЛРФЩ
  Расшифрованная фраза: РЪЦЯЫИЙОТЧ

Тест 7:
  Исходная фраза: АРМЯГЬУБГУ
  Зашифрованная фраза: БСНАДЭФВДФ
  Расшифрованная фраза: ЮПМЯГЪТБГТ

Тест 8:
  Исходная фраза: ЙМЫЯКЫДЭЮЕ
  Зашифрованная фраза: ОСАДПАЙВГК
  Расшифрованная фраза: МПЯГОЯИБЫЙ

Тест 9:
  Исходная фраза: ЗЬУЯГЯЮКИЩ
  Зашифрованная фраза: МБШДИДГПНЮ
  Расшифрованная фраза: ЙЮЧГЧГЫОМЫ

Тест 10:
  Исход

Как-то и с полносвязной сетью не очень получилось...возможно, делаю что-то не так, но пока не поняла что